In [ ]:
import polars as pl
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM, Bidirectional, GRU, SpatialDropout1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from wordcloud import WordCloud
import matplotlib.pyplot as plt

def load_and_preprocess_data(train_file, dev_file, test_file, max_features, max_words):
    # Load the data
    data_train = pl.read_csv(train_file)
    data_dev = pl.read_csv(dev_file)
    data_test = pl.read_csv(test_file)

    # Ensure the Tweet column is in string format
    data_train = data_train.with_column(pl.col("Tweet").cast(pl.Utf8))
    data_dev = data_dev.with_column(pl.col("Tweet").cast(pl.Utf8))
    data_test = data_test.with_column(pl.col("Tweet").cast(pl.Utf8))

    # Convert target variables to categorical
    y_train = to_categorical(data_train['Intensity_Class'].to_numpy())
    y_val = to_categorical(data_dev['Intensity_Class'].to_numpy())
    y_test = to_categorical(data_test['Intensity_Class'].to_numpy())

    # Prepare the training, validation, and test data
    X_train = data_train['Tweet'].to_list()
    X_val = data_dev['Tweet'].to_list()
    X_test = data_test['Tweet'].to_list()

    # Tokenize the text data
    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(X_train)

    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_val_seq = tokenizer.texts_to_sequences(X_val)
    X_test_seq = tokenizer.texts_to_sequences(X_test)

    # Pad sequences
    X_train_pad = pad_sequences(X_train_seq, maxlen=max_words)
    X_val_pad = pad_sequences(X_val_seq, maxlen=max_words)
    X_test_pad = pad_sequences(X_test_seq, maxlen=max_words)

    return X_train_pad, y_train, X_val_pad, y_val, X_test_pad, y_test

def create_wordcloud(text, title=None):
    wordcloud = WordCloud(
        background_color='black',
        max_words=200,
        max_font_size=40,
        scale=3,
        random_state=1
    ).generate(str(text))

    fig = plt.figure(1, figsize=(15, 15))
    plt.axis('off')
    if title:
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()

def build_and_compile_cnn_model(max_features, max_words, num_classes):
    model = Sequential()
    model.add(Embedding(max_features, 100, input_length=max_words))
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def build_and_compile_lstm_model(max_features, max_words, num_classes):
    model = Sequential()
    model.add(Embedding(max_features, 100, input_length=max_words))
    model.add(LSTM(64, dropout=0.4, return_sequences=True))
    model.add(LSTM(32, dropout=0.5, return_sequences=False))
    model.add(Dense(num_classes, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])
    return model

def build_and_compile_bgru_model(max_features, max_words, num_classes):
    model = Sequential()
    model.add(Embedding(max_features, 100, input_length=max_words))
    model.add(SpatialDropout1D(0.25))
    model.add(Bidirectional(GRU(64, dropout=0.4, return_sequences=True)))
    model.add(Bidirectional(GRU(32, dropout=0.5, return_sequences=False)))
    model.add(Dense(num_classes, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])
    return model

def main(train_file, dev_file, test_file, model_type='bgru'):
    # Define parameters
    max_features = 5000
    max_words = 100
    batch_size = 64
    epochs = 10
    num_classes = 4

    # Load and preprocess the data
    X_train_pad, y_train, X_val_pad, y_val, X_test_pad, y_test = load_and_preprocess_data(train_file, dev_file, test_file, max_features, max_words)

    # Build and compile the model
    if model_type == 'cnn':
        model = build_and_compile_cnn_model(max_features, max_words, num_classes)
    elif model_type == 'lstm':
        model = build_and_compile_lstm_model(max_features, max_words, num_classes)
    elif model_type == 'bgru':
        model = build_and_compile_bgru_model(max_features, max_words, num_classes)
    else:
        raise ValueError("Invalid model_type. Choose from 'cnn', 'lstm', or 'bgru'.")

    model.summary()

    # Define early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Train the model
    history = model.fit(X_train_pad, y_train, validation_data=(X_val_pad, y_val), epochs=epochs, batch_size=batch_size, verbose=1, callbacks=[early_stopping])

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test_pad, y_test)
    print(f'Test Loss: {loss}')
    print(f'Test Accuracy: {accuracy}')

if __name__ == '__main__':
    train_file = 'path/to/train.csv'
    dev_file = 'path/to/dev.csv'
    test_file = 'path/to/test.csv'
    main(train_file, dev_file, test_file, model_type='bgru')
